In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import seaborn as sns

In [2]:
raw = pd.read_csv('/Users/adrianhughes/Desktop/D_Ework/tensor_flow_proj/project_1/mlb_elo_latest.csv')
raw

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2,Winner,pred_winner,target1,pitcher_rate
0,2023-10-01,2023.0,0.0,NaN,STL,CIN,1499.567587,1485.123367,0.555101,0.444899,...,0.575820,0.424180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-01,2023.0,0.0,NaN,SEA,TEX,1516.277991,1535.226359,0.507269,0.492731,...,0.504610,0.495390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-01,2023.0,0.0,NaN,NYM,PHI,1506.248367,1523.132153,0.510240,0.489760,...,0.538668,0.461332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-10-01,2023.0,0.0,NaN,MIL,CHC,1502.093612,1498.788921,0.539214,0.460786,...,0.557476,0.442524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-10-01,2023.0,0.0,NaN,KCR,NYY,1423.429777,1541.893168,0.367310,0.632690,...,0.347503,0.652497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,2023-03-30,2023.0,0.0,NaN,CHC,MIL,1489.043510,1512.200668,0.501213,0.498787,...,0.457642,0.542358,1486.706994,1525.137403,4.0,0.0,1.0,1.0,0.0,1.0
2426,2023-03-30,2023.0,0.0,NaN,BOS,BAL,1498.037778,1498.610307,0.533664,0.466336,...,0.586818,0.413182,1507.649266,1485.860031,9.0,10.0,0.0,1.0,1.0,0.0
2427,2023-03-30,2023.0,0.0,NaN,WSN,ATL,1454.556936,1548.169577,0.401139,0.598861,...,0.322921,0.677079,1432.490158,1556.483969,2.0,7.0,0.0,0.0,0.0,1.0
2428,2023-03-30,2023.0,0.0,NaN,NYY,SFG,1543.060843,1514.395798,0.575216,0.424784,...,0.602331,0.397669,1558.039955,1504.922870,5.0,0.0,1.0,1.0,0.0,0.0


In [3]:
raw['date'] = pd.to_datetime(raw['date'])

In [4]:
raw['opp_code'] = raw['team2'].astype('category').cat.codes

In [5]:
raw['day_code'] = raw['date'].dt.dayofweek

In [6]:
raw['target'] = (raw['score1'] > raw['score2']).astype('int')

In [7]:
old = raw[raw['date'] < '2023-06-20']
new = raw[raw['date'] >= '2023-06-20']
old.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1087 entries, 1342 to 2428
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1087 non-null   datetime64[ns]
 1   season        1087 non-null   float64       
 2   neutral       1087 non-null   float64       
 3   playoff       0 non-null      float64       
 4   team1         1087 non-null   object        
 5   team2         1087 non-null   object        
 6   elo1_pre      1087 non-null   float64       
 7   elo2_pre      1087 non-null   float64       
 8   elo_prob1     1087 non-null   float64       
 9   elo_prob2     1087 non-null   float64       
 10  elo1_post     1084 non-null   float64       
 11  elo2_post     1084 non-null   float64       
 12  rating1_pre   1087 non-null   float64       
 13  rating2_pre   1087 non-null   float64       
 14  pitcher1      1086 non-null   object        
 15  pitcher2      1087 non-null   objec

In [8]:
train = old[old['date'] <= '2023-05-30'].copy()
test = old[old['date'] > '2023-05-30'].copy()

In [9]:
train.fillna(train.mean(), inplace=True)

/var/folders/kv/n3rwph3j7kl4qt0c74kt3s600000gn/T/ipykernel_25289/1059553085.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  train.fillna(train.mean(), inplace=True)
/var/folders/kv/n3rwph3j7kl4qt0c74kt3s600000gn/T/ipykernel_25289/1059553085.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train.fillna(train.mean(), inplace=True)


In [10]:
test.fillna(test.mean(), inplace=True)

/var/folders/kv/n3rwph3j7kl4qt0c74kt3s600000gn/T/ipykernel_25289/2145524900.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  test.fillna(test.mean(), inplace=True)
/var/folders/kv/n3rwph3j7kl4qt0c74kt3s600000gn/T/ipykernel_25289/2145524900.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test.fillna(test.mean(), inplace=True)


In [11]:
train.shape


(826, 33)

In [12]:
test.shape

(261, 33)

In [33]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=150, min_samples_split=10, random_state=1)

In [14]:
team1 = ['elo_prob1','rating_prob1','pitcher1_rgs','elo1_pre','rating1_pre','pitcher1_adj','rating_prob1']
team2 = ['elo_prob2','rating_prob2','pitcher2_rgs','elo2_pre','rating2_pre','pitcher2_adj','rating_prob2']

In [34]:
predictors = ['pitcher_rate','day_code','opp_code']
#predictors = ['elo_prob1','elo_prob2','rating_prob1','rating_prob2','pitcher1_rgs','pitcher2_rgs','day_code','opp_code']
#target = '

In [35]:
rf.fit(train[predictors], train['target1'])

RandomForestClassifier(min_samples_split=10, n_estimators=150, random_state=1)

In [36]:
preds = rf.predict(test[predictors])

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
acc = accuracy_score(test['target1'], preds)

In [39]:
acc

0.5862068965517241

In [40]:
combined = pd.DataFrame(dict(actual=test['target1'], prediction=preds))

In [41]:
pd.crosstab(index=combined['actual'], columns=combined['prediction'])

prediction,0.0,1.0
actual,,
0.0,112,38
1.0,70,41


In [42]:
from sklearn.metrics import precision_score

In [43]:
precision_score(test['target1'], preds)

0.5189873417721519

In [25]:
grouped_matches = old.groupby('team1')


In [26]:
group = grouped_matches.get_group('SFG')
group

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,rating2_post,score1,score2,Winner,pred_winner,target1,pitcher_rate,opp_code,day_code,target
1342,2023-06-19,2023.0,0.0,NaN,SFG,SDP,1536.571759,1523.888049,0.552597,0.447403,...,1538.087935,7.0,4.0,1.0,1.0,0.0,1.0,22,0.0,1
1447,2023-06-11,2023.0,0.0,NaN,SFG,CHC,1512.467128,1484.862012,0.573724,0.426276,...,1481.901011,13.0,3.0,1.0,1.0,0.0,0.0,5,6.0,1
1461,2023-06-10,2023.0,0.0,NaN,SFG,CHC,1515.670428,1481.658712,0.582718,0.417282,...,1484.513535,0.0,4.0,0.0,1.0,1.0,0.0,5,5.0,0
1475,2023-06-09,2023.0,0.0,NaN,SFG,CHC,1517.429100,1479.900040,0.587633,0.412367,...,1482.034901,2.0,3.0,0.0,1.0,1.0,0.0,5,4.0,0
1540,2023-06-04,2023.0,0.0,NaN,SFG,BAL,1515.151845,1516.815340,0.532101,0.467899,...,1505.771356,3.0,8.0,0.0,1.0,1.0,0.0,3,6.0,0
1554,2023-06-03,2023.0,0.0,NaN,SFG,BAL,1512.669594,1519.297592,0.524980,0.475020,...,1503.448089,4.0,0.0,1.0,1.0,0.0,0.0,3,5.0,1
1568,2023-06-02,2023.0,0.0,NaN,SFG,BAL,1514.214458,1517.752728,0.529413,0.470587,...,1505.263995,2.0,3.0,0.0,1.0,1.0,0.0,3,4.0,0
1597,2023-05-31,2023.0,0.0,NaN,SFG,PIT,1517.965462,1472.577318,0.598551,0.401449,...,1471.514889,4.0,9.0,0.0,1.0,1.0,0.0,21,2.0,0
1604,2023-05-30,2023.0,0.0,NaN,SFG,PIT,1519.784717,1470.758062,0.603573,0.396427,...,1468.987997,1.0,2.0,0.0,1.0,1.0,0.0,21,1.0,0
1622,2023-05-29,2023.0,0.0,NaN,SFG,PIT,1516.317670,1474.225109,0.593984,0.406016,...,1467.602841,14.0,4.0,1.0,1.0,0.0,0.0,21,0.0,1
